In [1]:
import pandas as pd
import numpy as np

import os

In [3]:
print(os.getcwd())
%cd ../../..

/Users/elizagoler/Documents/RecessionFertility/source/derived/county_gfrs
/Users/elizagoler/Documents/RecessionFertility


**I did the 1968 dictionary manually. Later dictionaries were generated by GPT given the codebook.**

In [32]:
# File structure defined based on SEER Data Dictionary

# Dictionary where key is the column name and value is the width
dict_1968 = {
    'year': 1,
    'gap1': 2,
    'gap2': 1,
    'gap3': 6,
    'record_type': 1, 
    'resident_status': 1, 
    'state': 2, 
    'county': 3,      # the two-space gap in the text file (junk column)
    'city': 3, 
    'population_size': 1, 
    'smsa': 3, 
    'metro_nonmetro_county_code': 1,
    'race_father': 1,
    'race_mother': 1,
    'race_child': 1,
    'race_recode1': 1,
    'race_recode2': 1,
    'sex': 1,
    'month_birth': 2,
    'gap4': 4,
    'age_mother': 2,
    'age_mother_recode1': 2,
    'age_mother_recode2': 1,
    'age_mother_recode3': 2,
    'gap5': 2,
    'nchildren_born_alive': 2,
    'nchildren_born_alive_recode1': 1,
    'nchildren_born_alive_recode2': 1,
    'gap6': 7,
    'attendant_at_birth': 1,
    'gestation_period': 1,
    'gestation_period_recode1': 2,
    'weight_birth': 1,
    'weight_birth_recode1': 4,
    'weight_birth_recode2': 2,
    'legitimacy': 1,
    'num_at_birth': 1,
    'num_at_birth_recode1': 1,
    'gap7': 2,
    'place_occurence': 5,
    'gap8': 3
}

# If you need the lists for read_fwf, you can get them like this:
widths_68 = list(dict_1968.values())
names_68 = list(dict_1968.keys())

file_path = 'datastore/raw/ICPSR_natality/data/1968-Data.txt'

icpsr_1968_natality_df = pd.read_fwf(
    file_path, 
    widths=widths_68, 
    names=names_68
)

# Drop the unused gap column
icpsr_1968_natality_df = icpsr_1968_natality_df.drop(columns=['gap1', 'gap2', 'gap3', 'gap4', 'gap5', 'gap6', 'gap7', 'gap8'])


icpsr_1968_natality_df.head()

,year,record_type,resident_status,state,county,city,population_size,smsa,metro_nonmetro_county_code,race_father,...,attendant_at_birth,gestation_period,gestation_period_recode1,weight_birth,weight_birth_recode1,weight_birth_recode2,legitimacy,num_at_birth,num_at_birth_recode1,place_occurence
0,8,1,1,1,20,999,9,0,2,1,...,1,4,6,3,2320,72,1,1,1,1020
1,8,1,1,1,1,999,9,0,2,1,...,1,4,6,2,8920,62,1,1,1,1001
2,8,1,1,1,1,999,9,0,2,2,...,1,3,43,1,9850,41,1,1,1,1001
3,8,2,2,1,26,999,9,0,2,1,...,1,4,6,3,2890,72,1,1,1,1001
4,8,1,1,1,1,999,9,0,2,9,...,3,4,6,3,7420,82,1,1,1,1001


In [ ]:
nchs_fips_crosswalk = pd.read_csv('datastore/raw/crosswalks/nchs_fips_crosswalk.csv')
icpsr_1968_natality_df.rename(columns={'state': 'nchs_state', 'county': 'nchs_county'}, inplace=True)

In [34]:
# Diagnostic: Check merge issues
print("Sample values from icpsr_1968_natality_df:")
print(icpsr_1968_natality_df[['nchs_state', 'nchs_county']].head(10))
print("\nUnique nchs_state values (first 20):")
print(icpsr_1968_natality_df['nchs_state'].unique()[:20])
print("\nUnique nchs_county values (first 20):")
print(icpsr_1968_natality_df['nchs_county'].unique()[:20])
print("\nAny 'nan' strings?", (icpsr_1968_natality_df['nchs_state'] == 'nan').any() or (icpsr_1968_natality_df['nchs_county'] == 'nan').any())

print("\n\nSample values from crosswalk:")
print(nchs_fips_crosswalk[['nchs_state', 'nchs_county']].head(10))
print("\nUnique nchs_state values (first 20):")
print(nchs_fips_crosswalk['nchs_state'].unique()[:20])
print("\nUnique nchs_county values (first 20):")
print(nchs_fips_crosswalk['nchs_county'].unique()[:20])

# Check for matching values
sample_state = icpsr_1968_natality_df['nchs_state'].iloc[0]
sample_county = icpsr_1968_natality_df['nchs_county'].iloc[0]
print(f"\n\nSample from natality: state='{sample_state}', county='{sample_county}'")
print(f"Type: state={type(sample_state)}, county={type(sample_county)}")
matches = nchs_fips_crosswalk[
    (nchs_fips_crosswalk['nchs_state'] == sample_state) & 
    (nchs_fips_crosswalk['nchs_county'] == sample_county)
]
print(f"Matches in crosswalk: {len(matches)}")
if len(matches) > 0:
    print(matches)

Sample values from icpsr_1968_natality_df:
   nchs_state  nchs_county
0           1           20
1           1            1
2           1            1
3           1           26
4           1            1
5           1            2
6           1            2
7           1            2
8           1            2
9           1            2

Unique nchs_state values (first 20):
[ 1 41 33 48 47 11 10 43 25 19 36 39 22 23 14 44  5  7 38 35]

Unique nchs_county values (first 20):
[ 20   1  26   2   3  57   4   5  28   6   7  21   8  32  29  61  17 339
   9  38]

Any 'nan' strings? False


Sample values from crosswalk:
   nchs_state  nchs_county
0           1            1
1           1            2
2           1            3
3           1            4
4           1            5
5           1            6
6           1            7
7           1            8
8           1            9
9           1           10

Unique nchs_state values (first 20):
[ 1  2  3  4  5  6  7  8 10 11 12 13 14 15 16

In [35]:
# Perform the merge
icpsr_1968_natality_df = icpsr_1968_natality_df.merge(
    nchs_fips_crosswalk[['nchs_state', 'nchs_county', 'fipsco']],
    on=['nchs_state', 'nchs_county'],
    how='left'
)

Checking results of the merge:

In [36]:
# Check merge results
print(f"Total rows in merged dataframe: {len(icpsr_1968_natality_df)}")
print(f"Rows with fipsco (matched): {icpsr_1968_natality_df['fipsco'].notna().sum()}")
print(f"Rows without fipsco (unmatched): {icpsr_1968_natality_df['fipsco'].isna().sum()}")
print(f"Match rate: {icpsr_1968_natality_df['fipsco'].notna().sum() / len(icpsr_1968_natality_df) * 100:.2f}%")

# Show some examples of unmatched rows
unmatched = icpsr_1968_natality_df[icpsr_1968_natality_df['fipsco'].isna()][['nchs_state', 'nchs_county']].drop_duplicates()
print(f"\nSample unmatched nchs_state/nchs_county combinations (first 10):")
print(unmatched.head(10))

# Check if any of these unmatched values exist in the crosswalk
if len(unmatched) > 0:
    sample_unmatched = unmatched.iloc[0]
    print(f"\nChecking if '{sample_unmatched['nchs_state']}'/'{sample_unmatched['nchs_county']}' exists in crosswalk:")
    exists = nchs_fips_crosswalk[
        (nchs_fips_crosswalk['nchs_state'] == sample_unmatched['nchs_state']) & 
        (nchs_fips_crosswalk['nchs_county'] == sample_unmatched['nchs_county'])
    ]
    print(f"Found: {len(exists)} rows")
    if len(exists) > 0:
        print(exists)

Total rows in merged dataframe: 1750782
Rows with fipsco (matched): 1715697
Rows without fipsco (unmatched): 35085
Match rate: 98.00%

Sample unmatched nchs_state/nchs_county combinations (first 10):
       nchs_state  nchs_county
104            47          339
1657           47          324
1710           47          336
8992            9            1
9003           47          174
9585           47          273
11464          47          276
15465          47          369
19552          47          318
36647          47          372

Checking if '47'/'339' exists in crosswalk:
Found: 0 rows


In [78]:
icpsr_1968_births_grouped = (
    icpsr_1968_natality_df
    .groupby(['year', 'fipsco'])
    .size()
    .reset_index(name='births')
)

print(icpsr_1968_births_grouped.head())

   year  fipsco  births
0     8    36.0   68151
1     8  1001.0     244
2     8  1003.0     501
3     8  1005.0     226
4     8  1007.0     125


In [21]:
natality_1995 = pd.read_csv('datastore/raw/nvss/data/natality1995us.csv')

/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_10621/2601039937.py:1: DtypeWarning: Columns (21,22,23,24,30,198,199) have mixed types. Specify dtype option on import or set low_memory=False.
  natality_1995 = pd.read_csv('datastore/raw/nvss/data/natality1995us.csv')


In [37]:
natality_1995.head()

,datayear,rectype,restatus,recwt,pldel,pldel3,birattnd,regnocc,divocc,stsubocc,...,alcuse,wtgn,obstrc,clabor,abnml,congan,educsmsa,apiflag,smsares,popsmas
0,1995,1,1,1,1,1,1,3,6,63,...,1,1,1,1,1,1,1,0,12,2
1,1995,1,1,1,1,1,1,3,6,63,...,1,1,1,1,1,1,1,0,12,2
2,1995,1,1,1,1,1,1,3,6,63,...,1,1,1,1,1,1,1,0,12,2
3,1995,1,1,1,1,1,1,3,6,63,...,1,1,1,1,1,1,1,0,12,2
4,1995,1,1,1,1,1,1,3,6,63,...,1,1,1,1,1,1,1,0,12,2


In [40]:
print([col for col in natality_1995.columns if 'fip' in col.lower()])

['stoccfip', 'cntocfip', 'stresfip', 'cntyrfip', 'placefip', 'smsarfip']


In [43]:
natality_1995_grouped = (
    natality_1995
    .groupby(['datayear', 'cntyrfip'])
    .size()
    .reset_index(name='births')
)
natality_1995_grouped.head()


,datayear,cntyrfip,births
0,1995,0,3423
1,1995,1015,1520
2,1995,1073,9372
3,1995,1089,3846
4,1995,1097,6093


In [58]:
natality_1995_grouped['datayear'] = natality_1995_grouped['datayear'].astype(str)
natality_1995_grouped['cntyrfip'] = natality_1995_grouped['cntyrfip'].astype(str)
natality_1995_grouped.dtypes

datayear    object
cntyrfip    object
births       int64
dtype: object

In [70]:
wonder_df = pd.read_csv('datastore/raw/cdc_wonder/data/cdc_wonder_1995-2002.csv')
#wonder_1995_df = wonder_df[wonder_df['year'] == 1995]
wonder_df = wonder_df.rename(columns=lambda x: x.strip().lower())
wonder_df = wonder_df.rename(columns={'year': 'datayear', 'county code': 'cntyrfip', 'births': 'births_wonder'})

# Remove decimal points for specified columns (if they exist in DataFrame)
for col in ['cntyrfip', 'datayear', 'year code', 'births_wonder']:
    if col in wonder_df.columns:
        # Convert to str, then to int (to drop decimals), then back to str (if needed)
        wonder_df[col] = wonder_df[col].apply(lambda x: str(int(float(x))) if pd.notnull(x) else x)

wonder_df.head()

wonder_df_1995 = wonder_df[wonder_df['datayear'] == '1995']

In [71]:
wonder_df_1995.head()
wonder_df_1995.dtypes


notes            object
county           object
cntyrfip         object
datayear         object
year code        object
births_wonder    object
dtype: object

In [72]:
natality_1995_grouped.head()
natality_1995_grouped.dtypes

datayear    object
cntyrfip    object
births       int64
dtype: object

In [73]:
merged_df = pd.merge(
    wonder_df_1995,
    natality_1995_grouped,
    on=['datayear', 'cntyrfip'],
    how='outer',
    suffixes=('_wonder', '_natality')
)
merged_df.head()


,notes,county,cntyrfip,datayear,year code,births_wonder,births
0,NaN,NaN,0,1995,NaN,NaN,3423
1,NaN,"Kent County, DE",10001,1995,1995,1738,1738
2,NaN,"New Castle County, DE",10003,1995,1995,6855,6855
3,NaN,"Sussex County, DE",10005,1995,1995,1673,1673
4,NaN,"Calhoun County, AL",1015,1995,1995,1520,1520


In [75]:
# Filter out rows where births_wonder is NaN
filtered_df = merged_df[merged_df['births_wonder'].notna()]

# Count rows where births_wonder and births are different
different_births_count = (filtered_df['births_wonder'] != filtered_df['births']).sum()

# Calculate the share of rows where births_wonder and births are different
total_rows = len(filtered_df)
if total_rows > 0:
    share_different = different_births_count / total_rows
else:
    share_different = float('nan')

different_births_count, share_different


(np.int64(507), np.float64(1.0))

Looks like only 1% of the data is different between CDC Wonder and the microdata aggregation method for a sample year in which I have both (1995). This means that aggregating the individual birth records seems like the right strat!